# Doctor Finder Multi-Agent System



In [ ]:
from kaggle_secrets import UserSecretsClient
import os
GOOGLE_API_KEY = UserSecretsClient().get_secret('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
print('API key loaded')

In [ ]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search, FunctionTool
from google.genai import types

retry_config = types.HttpRetryOptions(
    attempts=5, exp_base=7, initial_delay=1,
    http_status_codes=[429,500,503,504]
)
print('ADK imported')

## Session Initializer — FINAL VERSION

In [ ]:
session_init_agent = Agent(
    name='SessionInitializer',
    model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction='''
Extract symptoms and location from the user message.
Output ONLY in this exact format:
symptoms: <extracted symptoms>
location: <extracted location>
''',
    output_key='patient_info'
)
print('SessionInitializer ready')

## Parallel Specialty Guess Agents

## Aggregator

In [ ]:
guess_internal = Agent(
    name='InternalMedGuess', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction="Based on patient info {patient_info}, predict the most likely medical specialty.", output_key='guess_internal')

guess_emergency = Agent(
    name='EmergencyGuess', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction="Based on patient info {patient_info}, predict the most likely medical specialty.", output_key='guess_emergency')

guess_clinical = Agent(
    name='ClinicalGuess', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction="Based on patient info {patient_info}, predict the most likely medical specialty.", output_key='guess_clinical')

parallel_specialty = ParallelAgent(
    name='SpecialtyParallel', sub_agents=[guess_internal, guess_emergency, guess_clinical])

print('Parallel ready')

In [ ]:
specialty_aggregator = Agent(
    name='SpecialtyAggregator', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction='Choose best specialty among {guess_internal}, {guess_emergency}, {guess_clinical}.',
    output_key='final_specialty')
print('Aggregator ready')

## Doctor Search Agent

## Explanation Agent

In [ ]:
doctor_search_agent = Agent(
    name='DoctorSearchAgent', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction="""Patient info: {patient_info}. 
Use google_search tool to find doctors specializing in {final_specialty}. 
Extract location from patient_info.

Search for doctors with COMPLETE information including:
- Full name
- Specific clinic/hospital address
- Phone number or contact details
- Patient ratings/reviews if available
- Consultation fees if available

Perform thorough search to gather all available details.""",
    tools=[google_search], output_key='doctor_results')
print('Doctor search ready')

In [ ]:
explanation_agent = Agent(
    name='ExplanationAgent', 
    model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction='''Format the results as a well-organized markdown table:

**Patient Information:**
{patient_info}
Recommended Specialty: {final_specialty}

**Recommended Doctors:**
Create a table with columns: Doctor Name | Specialty | Location | Contact | Rating | Fee
Use the search results from: {doctor_results}

CRITICAL FILTERING RULES:
1. ONLY include doctors who have AT LEAST 3 of these details:
   - Specific clinic/hospital address (not just city name)
   - Phone number or contact information
   - Patient rating/reviews
   - Consultation fee

2. DO NOT include doctors with:
   - Only a name and city
   - Missing contact information
   - Hospital name as location without specific address

3. Show the most complete profiles first (doctors with 4+ details at top)

4. If a detail is truly not available after filtering, use "N/A"

5. If no doctors meet the criteria, display: "No doctors found with complete information. Please try a broader search or different location."

Format each doctor's information clearly with all available details.''',
    output_key='summary'
)
print('Explanation ready')

## Loop Refinement

In [ ]:
def exit_loop():
    return {'summary': 'Approved summary', 'status': 'approved'}

exit_tool = FunctionTool(exit_loop)

critic = Agent(
    name='Critic', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction='Review summary: {summary}. APPROVED or feedback.',
    output_key='critique')

refiner = Agent(
    name='Refiner', model=Gemini(model='gemini-2.5-flash-lite', retry_options=retry_config),
    instruction='If {critique} == APPROVED, call exit_loop. Otherwise refine {summary}.',
    tools=[exit_tool], output_key='summary')

loop = LoopAgent(name='RefinementLoop', sub_agents=[critic, refiner], max_iterations=1)
print('Loop ready')

## Final Pipeline

In [ ]:
pipeline = SequentialAgent(
    name='DoctorFinderPipeline',
    sub_agents=[
        session_init_agent,
        parallel_specialty,
        specialty_aggregator,
        doctor_search_agent,
        explanation_agent,
        loop
    ]
)

runner = InMemoryRunner(agent=pipeline)
print('Pipeline and runner ready')

## Run the System

In [ ]:
# Interactive input for symptoms and location
symptoms = input("Please describe your symptoms: ")
location = input("Please enter your location (city, country): ")

# Construct the query
user_query = f"symptoms: {symptoms}; location: {location}"
print(f"\n🔍 Searching for doctors based on:\n{user_query}\n")

In [ ]:
# Run the doctor finder system with user input
response = await runner.run_debug(user_query)

# Extract and display the final summary
final_summary = None
for event in response:
    if hasattr(event, 'author') and event.author == 'ExplanationAgent':
        if hasattr(event, 'actions') and event.actions and event.actions.state_delta:
            final_summary = event.actions.state_delta.get('summary')
            break

# Display the formatted result
if final_summary:
    from IPython.display import Markdown, display
    print("\n" + "="*80)
    print("📋 DOCTOR FINDER RESULTS")
    print("="*80 + "\n")
    display(Markdown(final_summary))
else:
    print("No summary found in response")